# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *
import scipy.sparse.linalg

In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(4*aLat) + aLat*(qnP1-qnM1)*V(qn)/(4*aLat) )
fermionNI=sp.expand( -(xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} - \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) - \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=4
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} - \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) - \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

c=-1
c2=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    return c + c2*qs[n]*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(8 - 2*\chi^{\dagger}_{0}*\chi_{1} + 2*\chi^{\dagger}_{0}*\chi_{3} - 2*\chi^{\dagger}_{1}*\chi_{0} - 2*\chi^{\dagger}_{1}*\chi_{2} - 2*\chi^{\dagger}_{2}*\chi_{1} - 2*\chi^{\dagger}_{2}*\chi_{3} + 2*\chi^{\dagger}_{3}*\chi_{0} - 2*\chi^{\dagger}_{3}*\chi_{2} + 2*p_{0}**2 + 2*p_{1}**2 + 2*p_{2}**2 + 2*p_{3}**2 + 4*q_{0} - 8*q_{0}*\chi^{\dagger}_{0}*\chi_{0} - q_{0}*q_{1}**2 - q_{0}*q_{2} + q_{0}*q_{3}**2 - 3*q_{0}**2 + q_{0}**2*q_{1} - q_{0}**2*q_{3} + 2*q_{0}**4 - 4.0*q_{1} + 8*q_{1}*\chi^{\dagger}_{1}*\chi_{1} + q_{1}*q_{0}**2 - q_{1}*q_{2}**2 - q_{1}*q_{3} - 3*q_{1}**2 - q_{1}**2*q_{0} + q_{1}**2*q_{2} + 2*q_{1}**4 + 4*q_{2} - 8*q_{2}*\chi^{\dagger}_{2}*\chi_{2} - q_{2}*q_{0} + q_{2}*q_{1}**2 - q_{2}*q_{3}**2 - 3*q_{2}**2 - q_{2}**2*q_{1} + q_{2}**2*q_{3} + 2*q_{2}**4 - 4.0*q_{3} + 8*q_{3}*\chi^{\dagger}_{3}*\chi_{3} - q_{3}*q_{0}**2 - q_{3}*q_{1} + q_{3}*q_{2}**2 - 3*q_{3}**2 + q_{3}**2*q_{0} - q_{3}**2*q_{2} + 2*q_{3}**4)/4

In [8]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

2 - \chi^{\dagger}_{0}*\chi_{1}/2 + \chi^{\dagger}_{0}*\chi_{3}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 - \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{3}/2 + \chi^{\dagger}_{3}*\chi_{0}/2 - \chi^{\dagger}_{3}*\chi_{2}/2 + 38613965*a^{\dagger}_{0}/54608393 - 131836323*a^{\dagger}_{0}*\chi^{\dagger}_{0}*\chi_{0}/93222358 - 3998607*a^{\dagger}_{0}*a^{\dagger}_{1}*a_{1}/45239074 - 3998607*a^{\dagger}_{0}*a^{\dagger}_{1}**2/45239074 - a^{\dagger}_{0}*a^{\dagger}_{2}/8 + 3998607*a^{\dagger}_{0}*a^{\dagger}_{3}*a_{3}/45239074 + 3998607*a^{\dagger}_{0}*a^{\dagger}_{3}**2/45239074 - a^{\dagger}_{0}*a_{0}/8 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}/8 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2/8 + 3998607*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{1}/45239074 - 3998607*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{3}/45239074 + 3998607*a^{\dagger}_{0}*a_{0}*a_{1}/45239074 - 3998607*a^{\dagger}_{0}*a_{0}*a_{3}/45239074 + a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}/8 + a^{\dagge

In [9]:
for cutoff in [2,3,4,5,6]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,which='SM')[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

/home/chrisculver/miniconda3/envs/QuantumComputing/lib/python3.9/site-packages/scipy/sparse/_sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


toHam timer took: 4.9252 seconds
evals timer took: 0.3721 seconds
Lambda =  2
[-0.08114224-7.7125151e-09j -0.08114344-1.1221614e-08j
 -0.0811431 +1.3928002e-08j -0.081143  +6.2864274e-09j
 -0.0811427 -3.8653525e-11j -0.08114223-3.2937084e-09j
 -0.19399971+1.4279579e-08j -0.08114161-3.0238201e-08j
 -0.19399908-5.2105857e-09j -0.08113953-6.8357249e-09j]

toHam timer took: 14.9548 seconds
evals timer took: 0.8875 seconds
Lambda =  3
[-0.08400058-6.53977850e-09j -0.08847921-7.01120939e-09j
 -0.08847296+2.28287504e-08j -0.08399567+1.75991275e-08j
  0.14862692+4.73297890e-09j  0.14861973-1.14471876e-10j
  0.1411114 -1.02414992e-08j  0.14110321+2.33750752e-09j
  0.14109749-2.01609895e-10j  0.1411003 +1.22008164e-08j]

toHam timer took: 66.0864 seconds
evals timer took: 1.2527 seconds
Lambda =  4
[0.16927268+1.9360641e-08j 0.17352998-9.3010648e-09j
 0.16926818-1.8716600e-08j 0.17352614+2.3039277e-08j
 0.2177026 -3.6872304e-08j 0.22037913-2.9696206e-08j
 0.21769457+8.0880174e-09j 0.22036749+6.3